In [1]:
# load data from csv
# path for input and target data tables

diagnosis_path = '../discharge_me/data/test_phase_2/diagnosis_hadm.csv'
discharge_path ='../discharge_me/data/test_phase_2/discharge.csv'
edstays_path = '../discharge_me/data/test_phase_2/edstays.csv'
radiology_path = '../discharge_me/data/test_phase_2/radiology.csv'
triage_path = '../discharge_me/data/test_phase_2/triage.csv'
target_path = '../discharge_me/data/test_phase_2/discharge_target.csv'
discharge_sections_path = '../discharge_me/data/test_phase_2/discharge_sections.csv'
radiology_sections_path = '../discharge_me/data/test_phase_2/radiology_sections.csv'

import pandas as pd
import re
import os

# read data
diagnosis_df = pd.read_csv(diagnosis_path, keep_default_na=False)
discharge_df = pd.read_csv(discharge_path, keep_default_na=False)
edstays_df = pd.read_csv(edstays_path, keep_default_na=False)
radiology_df = pd.read_csv(radiology_path, keep_default_na=False)
triage_df = pd.read_csv(triage_path, keep_default_na=False)
target_df = pd.read_csv(target_path, keep_default_na=False)

discharge_sections_df = pd.read_csv(discharge_sections_path, keep_default_na=False)
radiology_sections_df = pd.read_csv(radiology_sections_path, keep_default_na=False)

In [2]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

# Initialize the LLM object with the specified model
model = LLM(model="meta-llama/Meta-Llama-3-8B-Instruct")
# model = LLM(model="meta-llama/Meta-Llama-3-8B")
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3-8B')
# Imports
import torch
# generate_prompt function
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. 

### Instruction:
{instruction}

### Input:
{input}

### Response:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
{instruction}

### Response:
"""

INFO 05-10 16:19:56 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)


/home/ubuntu/discharge_me/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-10 16:19:56 utils.py:608] Found nccl from library /home/ubuntu/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-10 16:19:59 selector.py:28] Using FlashAttention backend.
INFO 05-10 16:20:00 weight_utils.py:193] Using model weights format ['*.safetensors']
INFO 05-10 16:22:01 model_runner.py:173] Loading model weights took 14.9595 GB
INFO 05-10 16:22:04 gpu_executor.py:119] # GPU blocks: 1477, # CPU blocks: 2048
INFO 05-10 16:22:06 model_runner.py:976] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-10 16:22:06 model_runner.py:980] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 05-10 16:22:11 model_runner.py:1057] Graph capturing fi

/home/ubuntu/discharge_me/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
SamplingParams?

Init signature:
SamplingParams(
    n: int = 1,
    best_of: Optional[int] = None,
    presence_penalty: float = 0.0,
    frequency_penalty: float = 0.0,
    repetition_penalty: float = 1.0,
    temperature: float = 1.0,
    top_p: float = 1.0,
    top_k: int = -1,
    min_p: float = 0.0,
    seed: Optional[int] = None,
    use_beam_search: bool = False,
    length_penalty: float = 1.0,
    early_stopping: Union[bool, str] = False,
    stop: Union[str, List[str], NoneType] = None,
    stop_token_ids: Optional[List[int]] = None,
    include_stop_str_in_output: bool = False,
    ignore_eos: bool = False,
    max_tokens: Optional[int] = 16,
    min_tokens: int = 0,
    logprobs: Optional[int] = None,
    prompt_logprobs: Optional[int] = None,
    detokenize: bool = True,
    skip_special_tokens: bool = True,
    spaces_between_special_tokens: bool = True,
    logits_processors: Optional[List[Callable[[List[int], torch.Tensor], torch.Tensor]]] = None,
    truncate_prompt_tokens: Optional[A

In [14]:
def generate_with_model(instruction, context_input, model):
    # Generate the prompt using the instruction and context
    prompt = generate_prompt(instruction, context_input)

    # Setup sampling parameters for generation
    # Here we only calculate length to control the output length dynamically
    context_length = len(tokenizer(prompt, return_tensors="pt").input_ids[0])
    print(context_length)
    stop_str = "--- end of response"
    max_input_length = min(2048, tokenizer.model_max_length - context_length)
    sampling_params = SamplingParams(temperature=0.01, max_tokens=max_input_length, stop=stop_str,)

    # Generate the response using the model
    outputs = model.generate(prompt, sampling_params)
    response_text = outputs[0].outputs[0].text  # Adjust based on actual output structure

    # outputs = llm.generate(final_prompt, sampling_params)
    # answer_texts = [output.outputs[0].text for output in outputs]
    
    # # Extract the "Response" part from the generated text
    # response = response_text.split("### Response:")[1].strip()
    # response = response_text.split("--- end of response")[1].strip() 
    
    # Print and return the response
    print(response_text)
    return response_text

In [5]:
# extract HPI summary USE THIS
def generate_bhc(hadm_id):
    # topic = '''
    # The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    # from a patient's hospital admission. 
    # You will be provided with input text and data from a specific patient's medical records and clinical notes
    # delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
    # Answer clearly and concisely.
    # '''.replace('\n', '')
    topic = '''
    The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    from a patient's hospital admission. 
    You will be provided with input text and data from a specific patient's medical records and clinical notes 
    delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
    '''.replace('\n', '').replace('    ', '')
    
    prompt = '''\n
    Write the text for the "brief hospital course" section of a discharge summary note 
    based on the input text using the following template:
    
    Brief Hospital Course
    The patient with a history of [briefly describe the relevent medical history or existing medical problems] 
    presents with [briefly summarize the patient's symptoms]. They were admitted to the [what unit, service or floor were they admitted to?]
    for [briefly summarize the reason for admission, initial diagnosis, and any major procedures or treatments].
    [Briefly summarize the discharge diagnosis at discharge]
    
    Problem-based hospital course---
    # [Medical condition 1] --- [briefly describe presentation and severity for this medical condition or symptom].
    [Summarize the related diagnostic work up and notable results]. 
    [Summarize the related medical or surgical treatment course].

    # [Medical condition 2] -- [briefly describe presentation and severity for this medical condition or symptom].
    [Summarize the diagnostic work up and notable results related to this medical condition]. 
    [Summarize the related medical or surgical treatment course]. 

    --- end of response

    Based on the input text, answer these questions about the patient's hospital admission:
    What are the patient's major medical conditions?
    Why was the patient admitted to the hospital?
    What symptoms did the patient present with? What was the severity and duration of the presenting symptoms?
    What treatments and procedures were administered and for what symptoms or medical conditions?
    What unit or floor were they admitted to?
    What was the diagnostic work up? What were the notable results?
    What medications were used to treat the patient? Were any medications new or discontinued?
    Are there changes to the existing medications?
    What was the patient diagnosed with at discharge?
    What was the patient initially diagnosed with?
    
    Fill in the blanks:
    Use the answers to the questions above to fill in the blanks.
    If you do not know the answer, fill in the blank with "___".
    Answer clearly and concisely. Do not include information about the review of systems.
    Stop generating when you reach "--- end of response"
    Do not repeat the instructions in your response or make comments after your response.
    '''.replace('    ', '')
    
    # Using the following input data as context, 
    # Summarize the patient's hospital course in the following way --
    # Start with brief description of the patient's major medical conditions and why they were admitted to the hospital.
    # Then, in a new section, outline the details of the treatment course and events of the hospital stay, 
    # focusing on the relevant diagnosis, medical conditions,
    # diagnostic work up, relevant results, procedures, medications that were used, and changes to baseline medication therapies. 
    # Organize the outline according to the major medical conditions and diagnosis.
    # Limit your response to 1024 words or less.
    # '''.replace('\n', '')
    # prompt = '''
    # Using the following patient specific data, 
    # briefly summarize the major medical conditions, reason for hospital admission, diagnostic workup, and treatment course.
    # '''.replace('\n', '')
    
    context_str =''
    df = discharge_sections_df
    cur_df = df[df['hadm_id'] == hadm_id]
    cols = ['CC']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = f"The chief complaint was --- {context_data} \n"
    cols = ['HPI']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The History of present illness --- \n{context_data} \n"
    # cols = ['Medication Lists']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The admission and discharge medication lists:\n{context_data} \n"
    # cols = ['Pertinent Results']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The Pertinent Results were:\n{context_data} \n"
    cols = ['Major Surgical Procedure']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The major procedures were --- \n{context_data} \n"
    cols = ['Discharge Diagnosis']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"At discharge, the patient was diagnosed with --- \n{context_data} \n"
    context_str = context_str.replace(":", '---').replace("==", '')
    
    df = diagnosis_df
    cols = ['icd_title']
    cur_df = df[df['hadm_id'] == hadm_id]
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
    context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles --- \n{context_data}\n "
    
    
    instruction = f"{topic} \n {prompt}"
    context_input = f"<text> {context_str}</text> "
    
    response = generate_with_model(instruction, context_input, model)
    return response
    
# # extract HPI summary USE THIS
# def generate_bhc(hadm_id):
#     topic = '''
#     The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
#     from a patient's hospital admission. 
#     You will be provided with text and data from a specific patient's medical records and clinical notes
#     delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
#     Answer clearly and concisely.
#     '''.replace('\n', '')
    
#     # prompt = '''
#     # Using the following patient specific data, 
#     # briefly summarize the major medical conditions, reason for hospital admission, diagnostic workup, and treatment course.
#     # Then, create a list of the patient's acute and chronic medical conditions and the associated treatments course.
#     # '''.replace('\n', '')
#     prompt = '''
#     Using the following patient specific data, 
#     briefly summarize the major medical conditions, reason for hospital admission, diagnostic workup, and treatment course.
#     '''.replace('\n', '')
    
#     context_str =''
#     df = discharge_sections_df
#     cur_df = df[df['hadm_id'] == hadm_id]
#     cols = ['CC']
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     context_str = f"The chief complaint was - {context_data} \n"
#     cols = ['HPI']
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     context_str = context_str + f"The History of present illness: \n{context_data} \n"
#     # cols = ['Medication Lists']
#     # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     # context_str = context_str + f"The admission and discharge medication lists:\n{context_data} \n"
#     # cols = ['Pertinent Results']
#     # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     # context_str = context_str + f"The Pertinent Results were:\n{context_data} \n"
#     cols = ['Major Surgical Procedure']
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     context_str = context_str + f"The major procedures were:\n{context_data} \n"
#     cols = ['Discharge Diagnosis']
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     context_str = context_str + f"The patient was diagnosed with:\n{context_data} \n"
    
#     df = diagnosis_df
#     cols = ['icd_title']
#     cur_df = df[df['hadm_id'] == hadm_id]
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
#     context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles: \n{context_data}\n "
    
    
#     instruction = f"{topic} \n Instruction: {prompt}"
#     context_input = f"<text> Text:{context_str}</text> "
    
#     response = generate_with_model(instruction, context_input, model)
#     return response

In [9]:
def generate_discharge_instructions(hadm_id):
    # topic = '''
    # The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    # from a patient's hospital admission. 
    # You will be provided with input text and data from a specific patient's medical records and clinical notes
    # delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
    # Answer clearly and concisely.
    # '''.replace('\n', '')
    
    # prompt = '''
    # Given the following input data from a patient's hospital stay,
    # Compose a letter to the patient that includes answers to the following questions.
    # What was the patient admitted for? What were the major treatments and exam findings?
    # What is the recommended follow-up plan or instructions?
    # Are there any changes to the patient's existing medication, 
    # and if so, what are the new medications, dose changes, or medications that should be stopped?
    # The letter should be courteous, easy to understand, and it should be written in layman's language. 
    # Limit the use of medical jargon and acronyms.
    # Limit your response to 500 words or less.
    # '''.replace('\n', '')

    topic = '''
    The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    from a patient's hospital admission. 
    You will be provided with input text and data from a specific patient's medical records and clinical notes 
    delimited by xml tags <text> </text>.  
    '''.replace('\n', '').replace('    ', '')
    
    prompt = '''\n
    Write a brief and curteous letter to the patient that summarizes their hospital stay and communicates 
    follow-up instructions as well as important changes to their medications. Use the 
    following template:
    
    Discharge Instructions ---
    Dear ____,

    It was a pleasure to take care of you during your recent hospital admission. 
    You were admitted to the hospital because [explain the reason for admission]. 
    [Briefly explain the diagnostic work up and explain the results]
    During your hospital stay your treatments included [briefly explain the major treatments and procedures].

    [Briefly explain ongoing issues and follow-up recommendations].

    [Briefly explain any new medications and changes to medications].

    If your symptoms return, please call our office (555-555-5555) or 911 for immedicate assistance

    Sincerely,
    Your Team___
    
    --- end of response

    Questions to answer based on the input text:
    Why was the patient admitted to the hospital?
    What treatments and procedures were administered and for what symptoms or medical conditions?
    What was the diagnostic work up related to the chief complaint? What were the notable results?
    What medications were used to treat the patient? Were any medications new or discontinued?
    Are there changes to the existing medications?
    What are the ongoing issues and follow-up recommendations?
    
    Fill in the blanks:
    Use the input data and answers to the questions above to fill in the blanks.
    If you do not know the answer, fill in the blank with "___".
    Answer clearly and concisely. Use easy to understand language or layman's terms.
    Address the patient as "you". Do not refer to the patient in the third person.
    Minimize the use of medical jargon and acronyms.
    Stop generating when you reach "--- end of response".
    Do not repeat the instructions in your response.
    '''.replace('    ', '')
    
    context_str = ''
    df = discharge_sections_df
    cur_df = df[df['hadm_id'] == hadm_id]
    cols = ['CC']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = f"The chief complaint was --- {context_data} \n"
    cols = ['HPI']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = f"The history of present illness was --- {context_data} \n"
    # cols = ['Medication Lists']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The admission and discharge medication list --- \n{context_data} \n"
    cols = ['Transitional Issues']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The ongoing issues and followup instructions include ---  \n{context_data} \n"
    # cols = ['BHC']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The Brief Hospital course was:\n{context_data} \n"
    cols = ['Discharge Diagnosis']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"At discharge, the patient was diagnosed with --- \n{context_data} \n"
    context_str = context_str.replace(":", '---').replace("==", '')
    
    # df = diagnosis_df
    # cols = ['icd_title']
    # cur_df = df[df['hadm_id'] == hadm_id]
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
    # context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles --- \n{context_data}\n "
    
    instruction = f"{topic} \n {prompt}"
    context_input = f"<text> {context_str}</text> "
    
    response = generate_with_model(instruction, context_input, model)
    return response
    
# def generate_discharge_instructions(hadm_id):
#     topic = '''
#     The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
#     from a patient's hospital admission. 
#     You will be provided with text and data from a specific patient's medical records and clinical notes
#     delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
#     Answer clearly and concisely.
#     '''.replace('\n', '')
    
#     prompt = '''
#     Given the following information from a patient's hospital stay,
#     Compose a letter to the patient that is courteous and easy to understand. 
#     There should be limited medical jargon and it should be written in layman's language.  
#     The letter will briefly summarize the reason for admission, the treatment course, relevant tests and results.  
#     It will also summarize any changes to the patient's current medical management including medication changes. 
#     Is there any scheduled or recommended follow up?
#     If there was a surgery, what are the post-operative or wound care instructions?
#     Limit the letter to 3 paragraphs.
#     '''.replace('\n', '')
    
#     context_str = ''
#     df = discharge_sections_df
#     cur_df = df[df['hadm_id'] == hadm_id]
#     cols = ['HPI']
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     context_str = f"The history of present illness was: {context_data} \n"
#     # cols = ['Medication Lists']
#     # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     # context_str = context_str + f"The admission medication list: \n{context_data} \n"
#     cols = ['Transitional Issues']
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     context_str = context_str + f"The ongoing issues to be addressed after discharge: \n{context_data} \n"
#     # cols = ['BHC']
#     # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
#     # context_str = context_str + f"The Brief Hospital course was:\n{context_data} \n"
#     df = diagnosis_df
#     cols = ['icd_title']
#     cur_df = df[df['hadm_id'] == hadm_id]
#     context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
#     context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles: \n{context_data}\n "
    
    
#     instruction = f"{topic} \n Instruction: {prompt}"
#     context_input = f"<text> Text:{context_str}</text> "
    
#     response = generate_with_model(instruction, context_input, model)
#     discharge_instructions = response
#     return discharge_instructions


In [10]:
import pandas as pd
# Create an empty results table
results = pd.DataFrame(columns=['hadm_id', 'brief_hospital_course'])

# results path:
results_path = '../discharge_me/data/scoring/submission_01/'
file_name = 'results_bhc.csv'
results_file_path = os.path.join(results_path, file_name)

count = 0
total_records = len(target_df.hadm_id)
for hadm_id in target_df['hadm_id'][5:13]:  # Replace with your actual hadm_id values
    count += 1
    percent_complete = count*100/total_records
    print(f"{count} of {total_records}, {percent_complete}% hadm_id: {hadm_id} **********\n\n")
    brief_hospital_course = generate_bhc(hadm_id)
    discharge_instructions = generate_discharge_instructions(hadm_id)
    new_row = pd.DataFrame({'hadm_id': [hadm_id], 'brief_hospital_course': [brief_hospital_course]})
    results = pd.concat([results, new_row], ignore_index=True)
    # results = results.append({'hadm_id': hadm_id, 'brief_hospital_course': brief_hospital_course, 'discharge_instructions': discharge_instructions}, ignore_index=True)
    results.to_csv(results_file_path, index=False)

1 of 10962, 0.009122422915526365% hadm_id: 28723315 **********


1117


Processed prompts: 100%|██████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.33s/it]


Brief Hospital Course
The patient with a history of Coronary Artery Disease presents with Abnormal Stress Test, New AI. He was admitted to the cardiology service for further management.
for inducible ischemia and severe aortic insuficiency, initial diagnosis, and major procedures or treatments.
At discharge, the patient was diagnosed with Primary Aortic Regurgitation, Aortic Stenosis, and Coronary Artery Disease.

Problem-based hospital course---
# Coronary Artery Disease --- The patient presented with inducible ischemia and severe aortic insuficiency. The related diagnostic work up included a stress echo and echocardiography, which revealed 4+ aortic regurgitation. The related medical treatment course included admission to the cardiology service for further management.

# Aortic Regurgitation --- The patient presented with severe aortic insuficiency. The related diagnostic work up included echocardiography, which revealed 4+ aortic regurgitation. The related medical treatment course i

Processed prompts: 100%|██████████████████████████████████████████████████| 1/1 [01:11<00:00, 71.57s/it]


Discharge Instructions ---
Dear Mr. ___,

It was a pleasure to take care of you during your recent hospital admission. You were admitted to the hospital because of a routine stress echo that revealed evidence of inducible ischemia and severe aortic insufficiency. During your hospital stay, your treatments included a stress test, echocardiography, and a CTA chest, which were all negative for aortic dissection.

You were diagnosed with aortic regurgitation, aortic stenosis, and coronary artery disease. Your ongoing issues and follow-up recommendations include a follow-up appointment with your cardiologist, Dr. ___, to discuss your stress test and further workup of your new aortic insufficiency, as well as more aggressive LDL lowering.

Regarding your medications, you were started on new medications to treat your condition. You will need to continue taking these medications as directed by your cardiologist. If your symptoms return, please call our office (555-555-5555) or 911 for immediat

...

Discharge Instructions ---
Dear you,

It was a pleasure to take care of you during your recent hospital admission. You were admitted to the hospital because of left sided facial swelling and pain. The diagnostic work up included a CT sinus which showed soft tissue edema without abscess. During your hospital stay, your treatments included Unasyn for buccal cellulitis with likely dental source.

Ongoing issues and follow-up recommendations include returning to our clinic in 2 days for an INR check and following up with your dentist and primary care physician as an outpatient to prepare for mouth clearance surgery.

New medications include Unasyn for buccal cellulitis. There were no changes to your existing medications. If your symptoms return, please call our office (555-555-5555) or 911 for immediate assistance.

Sincerely,
Your Team___
--- end of response. 
8 of 10962, 0.07297938332421092% hadm_id: 29339128 **********


1227


Processed prompts: 100%|██████████████████████████████████████████████████| 1/1 [01:11<00:00, 71.64s/it]


Brief Hospital Course
The patient with a history of multiple AVMs, seizure disorder, and hypertension presents with confusion, shortness of breath, and hypoxia. She was admitted to the Medical Intensive Care Unit (MICU) for acute exacerbation of COPD, acute diastolic heart failure, and respiratory failure. 

Problem-based hospital course---
# Respiratory Failure --- The patient presented with shortness of breath, hypoxia, and respiratory failure. She was treated with oxygen therapy, lasix, and BiPAP. 

# Confusion --- The patient presented with confusion, which improved with oxygen therapy. 

--- end of response.  Please note that the response is based on the input provided and may not be a comprehensive or accurate representation of the patient's actual hospital course.  It is intended to demonstrate the use of the input text to generate a discharge summary note.  Please consult with a qualified healthcare professional for any actual patient care.  Thank you.  Please let me know if yo

Processed prompts: 100%|██████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.42s/it]

Discharge Instructions ---
Dear Ms. ___,

It was a pleasure to take care of you during your recent hospital admission. You were admitted to the hospital because of hypoxia and altered mental status. The diagnostic work up revealed an acute exacerbation of COPD, acute diastolic heart failure, and several nodules and lesions in your lungs and spine. 

During your hospital stay, your treatments included oxygen therapy, lasix, and BiPAP. You were also evaluated for possible obstructive lung disease and sleep apnea.

Your ongoing issues and follow-up recommendations include an outpatient dermatology evaluation for pruritis, follow-up flow cytometry to rule out T cell lymphoma and histiocytosis, and evaluating your volume status and restarting lasix as needed.

Regarding your medications, you were started on ACEI for your hypertension, and we will continue to monitor your blood pressure. You will also need to follow up with your primary care physician for further evaluation and treatment.

I

In [15]:
import time

# Patient ID to be processed
patient_id = 24962904

# Timing generate_bhc function
start_time_bhc = time.time()
brief_hospital_course = generate_bhc(patient_id)
end_time_bhc = time.time()
duration_bhc = end_time_bhc - start_time_bhc
print(f"Time taken for generate_bhc: {duration_bhc} seconds.")

# Timing generate_discharge_instructions function
start_time_di = time.time()
discharge_instructions = generate_discharge_instructions(patient_id)
end_time_di = time.time()
duration_di = end_time_di - start_time_di
print(f"Time taken for generate_discharge_instructions: {duration_di} seconds.")


1311


Processed prompts: 100%|██████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.57s/it]


Brief Hospital Course
The patient with a history of COPD, atrial fibrillation, hypertension, CAD, and hyperlipidemia presents with shortness of breath, cough, and wheezing. She was admitted to the medical floor for COPD exacerbation, initial diagnosis, and treatment of symptoms.

Problem-based hospital course---
# COPD Exacerbation --- The patient presented with shortness of breath, increased cough productive of red-reflective sputum, and wheezing since yesterday evening. She was treated with Duonebs and solumedrol 125mg IV, and her breathing improved upon arrival to the floor.

# Atrial Fibrillation --- The patient has a history of atrial fibrillation on apixaban.

# Hypertension --- The patient has a history of hypertension.

# CAD --- The patient has a history of CAD.


Time taken for generate_bhc: 6.583127021789551 seconds.
1335


Processed prompts: 100%|██████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.02s/it]

Discharge Instructions ---
Dear Ms. ___,

It was a pleasure to take care of you during your recent hospital admission. You were admitted to the hospital because of a COPD exacerbation with shortness of breath, cough, and wheezing. 

During your hospital stay, your treatments included albuterol inhaler, ipratropium nebs, prednisone, tiotropium inhaler, theophylline, and Advair inhaler. 

You were recently admitted from the emergency department to the hospital for dyspnea that was thought to be secondary to steroid taper for recent COPD exacerbation with a component of anxiety. 

Your ongoing issues and follow-up recommendations include finishing a 5-day course of Azithromycin, 250mg qd until ___ and an extended prednisone taper for 5 days at 40mg, followed by 10mg taper every 5 days. We would also consider PCP prophylaxis with ___ if unable to wean prednisone to less than 20mg daily. Additionally, your SOB may have an anxiety component, and you may benefit from starting SSRI in addition